In [164]:
import numpy as np
import pandas as pd
from math import isnan
import re

In [165]:
from ape_producer.geom_reader import create_geom_df
from ape_producer.parse_report import make_cov_objects
from ape_producer.sign_conventions import signConventions, square_coordinates

In [166]:
# ipython magic to autoreload modules for development
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [167]:
#options
outName = 'Test'
isDT = True
isData = False
is3DOF = False
outname = '_test'
xmlFileAlign = 'data/run3MC_DT_large_01.xml'
reportfile = 'data/run3MC_DT_large_01_report.py'
xmlFileIdealGeom = 'Geom/muonGeometry_IDEAL_AllZeroes.Ape6x6.StdTags.746p3.DBv2.xml'
debug = True

In [168]:
xmlFileAlign = 'csc_files/data_CSC-1100-110001_2018BC-ReReco_CMSSW102_JSON-318828-319993_dataRun2_MuAl_SeptRereco_v1_01.xml'
    
reportfile = 'csc_files/data_CSC-1100-110001_2018BC-ReReco_CMSSW102_JSON-318828-319993_dataRun2_MuAl_SeptRereco_v1_01_report.py'

In [169]:
exec(open(reportfile).read())
aligned_geom_dt, aligned_geom_csc  = create_geom_df(xmlFileAlign)
if not isData:
    ideal_geom_dt, ideal_geom_csc = create_geom_df(xmlFileIdealGeom)

In [235]:
dt_cov_df, csc_cov_df = make_cov_objects(reports, debug=False)

In [171]:
def average_covMatrix(df, **kwargs):
    df = df[df.status=="PASS"]
    if df.shape[0] == 0: return np.full(36, 0).reshape(6,6)
    for k, v in kwargs.items():
        df = df[df[k]==v]
    return df.covMatrix.mean()

In [172]:
def average_sectors(df, dont_average_over=[]):
    average_over = np.setdiff1d(df.columns,dont_average_over)
    unique_items = np.unique(df[average_over], axis=0)
    unique_items_dicts = [{k:v for k, v in zip(average_over, l)} for l in unique_items]
    average_cov = [ {**item_dict, "covMatrix":average_covMatrix(df, **item_dict)} for item_dict in unique_items_dicts]
    return pd.DataFrame(average_cov)

In [230]:
average_csc_cov = average_sectors(csc_cov_df, dont_average_over=['endcap', 'sector', 'status', 'covMatrix'])

In [231]:
average_dt_cov = average_sectors(dt_cov_df, dont_average_over=[ 'sector', 'status', 'covMatrix'])

In [250]:
def sign_zero_positive(x): return 2*(x >= 0) - 1

def csc_ape_xml_string(df, average_csc_cov):
    df['sort_by'] = (df.endcap*1e4 + df.station*1e3 + df.ring*1e2 + df.sector)
    df.sort_values(inplace = True, by=['sort_by'])
    ape_string = ""
    ape_string+=("<?xml version=\"1.0\" encoding=\"UTF-8\"?>\n")
    ape_string+=("<?xml-stylesheet type=\"text/xml\" href=\"MuonAlignment.xsl\"?>\n")
    ape_string+=("<MuonAlignment>\n\n")
    for i, (endcap, station, ring, sector) in df[['endcap', 'station', 'ring', 'sector']].iterrows():
        print(endcap, station, ring, sector)
        cov_matrix = average_csc_cov[(average_csc_cov.station==station) & (average_csc_cov.ring==ring)].covMatrix.iloc[0]
        cov_matrix = cov_matrix.reshape(-1)
        ape_string+=("<operation>\n")
        ape_string+=("  <CSCChamber endcap=\"%s\" station=\"%s\" ring=\"%s\" chamber=\"%s\" />\n" % (endcap,station,ring,sector) )
        ape_string+=("  <setape xx='{:10f}' xy='{:10f}' xz='{:10f}' yy='{:10f}' yz='{:10f}' zz='{:10f}' xa='{:10f}' xb='{:10f}' xc='{:10f}' ya='{:10f}' yb='{:10f}' yc='{:10f}' za='{:10f}' zb='{:10f}' zc='{:10f}' aa='{:10f}' ab='{:10f}' ac='{:10f}' bb='{:10f}' bc='{:10f}' cc='{:10f}' />\n".format(*cov_matrix))
        ape_string+=("</operation>\n\n")
    ape_string+=("</MuonAlignment>")
    return ape_string

In [251]:
def dt_ape_xml_string(df, average_dt_cov):
    #sort for ease of reading output
    df['sort_by'] = -sign_zero_positive(df.wheel) * (abs(df.wheel*1e3) - df.station*1e2 - df.sector)
    df.sort_values(inplace = True, by=['sort_by'])
    #write header
    ape_string = ""
    ape_string+=("<?xml version=\"1.0\" encoding=\"UTF-8\"?>\n")
    ape_string+=("<?xml-stylesheet type=\"text/xml\" href=\"MuonAlignment.xsl\"?>\n")
    ape_string+=("<MuonAlignment>\n\n")
    #write body
    for i, (wheel, station, sector) in df[['wheel', 'station', 'sector']].iterrows():
        print(wheel, station, sector)
        cov_matrix = average_dt_cov[(average_dt_cov.wheel==wheel) & (average_dt_cov.station==station)].covMatrix.iloc[0]
        cov_matrix = cov_matrix.reshape(-1)
        ape_string+=("<operation>\n")
        ape_string+=("  <DTChamber wheel=\"%s\" station=\"%s\" sector=\"%s\" />\n" % (wheel, station, sector) )
        ape_string+=("  <setape xx='{:10f}' xy='{:10f}' xz='{:10f}' yy='{:10f}' yz='{:10f}' zz='{:10f}' xa='{:10f}' xb='{:10f}' xc='{:10f}' ya='{:10f}' yb='{:10f}' yc='{:10f}' za='{:10f}' zb='{:10f}' zc='{:10f}' aa='{:10f}' ab='{:10f}' ac='{:10f}' bb='{:10f}' bc='{:10f}' cc='{:10f}' />\n".format(*cov_matrix))
        ape_string+=("</operation>\n\n")
    #close muon alignment
    ape_string+=("</MuonAlignment>")
    return ape_string

In [252]:
with open('dt'+outname+'.xml', 'w') as f:
    print(dt_ape_xml_string(dt_cov_df, average_dt_cov), file=f)

2 1 1
2 1 2
2 1 3
2 1 4
2 1 5
2 1 6
2 1 7
2 1 8
2 1 9
2 1 10
2 1 11
2 1 12
2 2 1
2 2 2
2 2 3
2 2 4
2 2 5
2 2 6
2 2 7
2 2 8
2 2 9
2 2 10
2 2 11
2 2 12
2 3 1
2 3 2
2 3 3
2 3 4
2 3 5
2 3 6
2 3 7
2 3 8
2 3 9
2 3 10
2 3 11
2 3 12
2 4 1
2 4 2
2 4 3
2 4 4
2 4 5
2 4 6
2 4 7
2 4 8
2 4 9
2 4 10
2 4 11
2 4 12
2 4 13
2 4 14
1 1 1
1 1 2
1 1 3
1 1 4
1 1 5
1 1 6
1 1 7
1 1 8
1 1 9
1 1 10
1 1 11
1 1 12
1 2 1
1 2 2
1 2 3
1 2 4
1 2 5
1 2 6
1 2 7
1 2 8
1 2 9
1 2 10
1 2 11
1 2 12
1 3 1
1 3 2
1 3 3
1 3 4
1 3 5
1 3 6
1 3 7
1 3 8
1 3 9
1 3 10
1 3 11
1 3 12
1 4 1
1 4 2
1 4 3
1 4 4
1 4 5
1 4 6
1 4 7
1 4 8
1 4 9
1 4 10
1 4 11
1 4 12
1 4 13
1 4 14
0 1 1
0 1 2
0 1 3
0 1 4
0 1 5
0 1 6
0 1 7
0 1 8
0 1 9
0 1 10
0 1 11
0 1 12
0 2 1
0 2 2
0 2 3
0 2 4
0 2 5
0 2 6
0 2 7
0 2 8
0 2 9
0 2 10
0 2 11
0 2 12
0 3 1
0 3 2
0 3 3
0 3 4
0 3 5
0 3 6
0 3 7
0 3 8
0 3 9
0 3 10
0 3 11
0 3 12
0 4 1
0 4 2
0 4 3
0 4 4
0 4 5
0 4 6
0 4 7
0 4 8
0 4 9
0 4 10
0 4 11
0 4 12
0 4 13
0 4 14
-1 4 14
-1 4 13
-1 4 12
-1 4 11
-1 4 10
-1 4 9
-1 4 8
-1 4

In [253]:
with open('csc'+outname+'.xml', 'w') as f:
    print(csc_ape_xml_string(csc_cov_df, average_csc_cov), file=f)

1 1 1 1
1 1 1 2
1 1 1 3
1 1 1 4
1 1 1 5
1 1 1 6
1 1 1 7
1 1 1 8
1 1 1 9
1 1 1 10
1 1 1 11
1 1 1 12
1 1 1 13
1 1 1 14
1 1 1 15
1 1 1 16
1 1 1 17
1 1 1 18
1 1 1 19
1 1 1 20
1 1 1 21
1 1 1 22
1 1 1 23
1 1 1 24
1 1 1 25
1 1 1 26
1 1 1 27
1 1 1 28
1 1 1 29
1 1 1 30
1 1 1 31
1 1 1 32
1 1 1 33
1 1 1 34
1 1 1 35
1 1 1 36
1 1 2 1
1 1 2 2
1 1 2 3
1 1 2 4
1 1 2 5
1 1 2 6
1 1 2 7
1 1 2 8
1 1 2 9
1 1 2 10
1 1 2 11
1 1 2 12
1 1 2 13
1 1 2 14
1 1 2 15
1 1 2 16
1 1 2 17
1 1 2 18
1 1 2 19
1 1 2 20
1 1 2 21
1 1 2 22
1 1 2 23
1 1 2 24
1 1 2 25
1 1 2 26
1 1 2 27
1 1 2 28
1 1 2 29
1 1 2 30
1 1 2 31
1 1 2 32
1 1 2 33
1 1 2 34
1 1 2 35
1 1 2 36
1 1 3 1
1 1 3 2
1 1 3 3
1 1 3 4
1 1 3 5
1 1 3 6
1 1 3 7
1 1 3 8
1 1 3 9
1 1 3 10
1 1 3 11
1 1 3 12
1 1 3 13
1 1 3 14
1 1 3 15
1 1 3 16
1 1 3 17
1 1 3 18
1 1 3 19
1 1 3 20
1 1 3 21
1 1 3 22
1 1 3 23
1 1 3 24
1 1 3 25
1 1 3 26
1 1 3 27
1 1 3 28
1 1 3 29
1 1 3 30
1 1 3 31
1 1 3 32
1 1 3 33
1 1 3 34
1 1 3 35
1 1 3 36
1 1 4 1
1 1 4 2
1 1 4 3
1 1 4 4
1 1 4 5
1 1 4 6
1 1 4 7

In [ ]:
csc_cov_df[(csc_cov_df.endcap==1) & 
          (csc_cov_df.disk==1) & 
          (csc_cov_df.ring==1) & 
          (csc_cov_df.sector==1) ]

In [17]:
#why is 4/1 the same?
csc_cov_df[(csc_cov_df.endcap==1) & 
          (csc_cov_df.disk==1) & 
          (csc_cov_df.ring==4) & 
          (csc_cov_df.sector==1) ]

,endcap,disk,ring,sector,status,covMatrix
108,1,1,4,1,PASS,"[[4.15641e-05, -1.18077e-05, 2.19617e-07, 5.83..."
